* SQL mini project  - Tier 2 of the case study


* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */


SELECT name
FROM Facilities
WHERE membercost > 0;


/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT( * ) AS free_facility_count
FROM Facilities
WHERE membercost =0;


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost >0
AND membercost < 0.2 * monthlymaintenance;




/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM Facilities
WHERE facid IN (1, 5);



/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */


SELECT 
  name,
  monthlymaintenance,
  CASE 
    WHEN monthlymaintenance > 100 THEN 'expensive'
    ELSE 'cheap'
  END AS cost_label
FROM Facilities;


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT firstname, surname
FROM Members
WHERE joindate = (
  SELECT MAX(joindate)
  FROM Members
);


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */


SELECT DISTINCT 
  f.name AS court_name,
  m.firstname,
  m.surname
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY m.firstname, m.surname;


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


SELECT f.name, b.memid, b.slots, f.membercost, f.guestcost
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
WHERE b.starttime LIKE '2012-09-14%';


/* Q9: This time, produce the same result as in Q8, but using a subquery. */


SELECT 
  facility_name,
  member_firstname,
  member_surname,
  cost
FROM (
  SELECT 
    f.name AS facility_name,
    IF(b.memid = 0, 'Guest', m.firstname) AS member_firstname,
    IF(b.memid = 0, '', m.surname) AS member_surname,
    IF(b.memid = 0, f.guestcost * b.slots, f.membercost * b.slots) AS cost,
    b.starttime
  FROM Bookings b
  JOIN Facilities f ON b.facid = f.facid
  LEFT JOIN Members m ON b.memid = m.memid
) AS booking_costs
WHERE starttime LIKE '2012-09-14%'
  AND cost > 30
ORDER BY cost DESC;




In [74]:
!pip install ipython-sql


In [76]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [77]:
%sql sqlite:///country_club.db


In [78]:
%%sql
SELECT name FROM Facilities LIMIT 5;


 * sqlite:///country_club.db
Done.


name
Tennis Court 1
Tennis Court 2
Badminton Court
Table Tennis
Massage Room 1


In [79]:
import sqlite3
import pandas as pd

# Connect to or create SQLite DB
conn = sqlite3.connect("country_club.db")

# Re-import the fixed CSVs (assume they now have correct headers)
facilities_df = pd.read_csv("Facilities.csv")
bookings_df = pd.read_csv("Bookings.csv")
members_df = pd.read_csv("Members.csv")

# Overwrite tables with correct structure
facilities_df.to_sql("Facilities", conn, if_exists="replace", index=False)
bookings_df.to_sql("Bookings", conn, if_exists="replace", index=False)
members_df.to_sql("Members", conn, if_exists="replace", index=False)

conn.close()


In [80]:
conn = sqlite3.connect('country_club.db')
cursor = conn.cursor()

for table in ['Facilities', 'Bookings', 'Members']:
    print(f"Table: {table}")
    cursor.execute(f"PRAGMA table_info({table});")
    for col in cursor.fetchall():
        print(col)
    print("\n")


Table: Facilities
(0, 'facid', 'INTEGER', 0, None, 0)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'membercost', 'REAL', 0, None, 0)
(3, 'guestcost', 'REAL', 0, None, 0)
(4, 'initialoutlay', 'INTEGER', 0, None, 0)
(5, 'momthlymaintenance', 'INTEGER', 0, None, 0)


Table: Bookings
(0, 'bookid', 'INTEGER', 0, None, 0)
(1, 'facid', 'INTEGER', 0, None, 0)
(2, 'memid', 'INTEGER', 0, None, 0)
(3, 'starttime', 'TEXT', 0, None, 0)
(4, 'slots', 'INTEGER', 0, None, 0)


Table: Members
(0, 'memid', 'INTEGER', 0, None, 0)
(1, 'surname', 'TEXT', 0, None, 0)
(2, 'firstname', 'TEXT', 0, None, 0)
(3, 'address', 'TEXT', 0, None, 0)
(4, 'zipcode', 'INTEGER', 0, None, 0)
(5, 'telephone', 'TEXT', 0, None, 0)
(6, 'recommendedby', 'REAL', 0, None, 0)
(7, 'joindate', 'TEXT', 0, None, 0)




In [82]:
#Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!
%%sql
SELECT 
    f.name AS facility_name,
    SUM(
        CASE 
            WHEN b.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots
        END
    ) AS total_revenue
FROM 
    Bookings b
JOIN 
    Facilities f ON b.facid = f.facid
GROUP BY 
    f.name
HAVING 
    total_revenue < 1000
ORDER BY 
    total_revenue;


 * sqlite:///country_club.db
Done.


facility_name,total_revenue
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0


In [85]:
#Produce a report of members and who recommended them in alphabetic surname,firstname order 

%%sql
SELECT 
    m.surname || ', ' || m.firstname AS member_name,
    r.surname || ', ' || r.firstname AS recommended_by
FROM 
    Members m
LEFT JOIN 
    Members r ON m.recommendedby = r.memid
ORDER BY 
    m.surname, m.firstname;


 * sqlite:///country_club.db
Done.


member_name,recommended_by
"Bader, Florence","Stibbons, Ponder"
"Baker, Anne","Stibbons, Ponder"
"Baker, Timothy","Farrell, Jemima"
"Boothe, Tim","Rownam, Tim"
"Butters, Gerald","Smith, Darren"
"Coplin, Joan","Baker, Timothy"
"Crumpet, Erica","Smith, Tracy"
"Dare, Nancy","Joplette, Janice"
"Farrell, David",None
"Farrell, Jemima",None


In [86]:
%load_ext sql
%sql sqlite:///country_club.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [88]:
#Find the facilities with their usage by member, but not guests 

%%sql
SELECT 
    f.name AS facility_name,
    SUM(b.slots) AS total_usage_by_members
FROM 
    Bookings b
JOIN 
    Facilities f ON b.facid = f.facid
WHERE 
    b.memid != 0
GROUP BY 
    f.name
ORDER BY 
    total_usage_by_members DESC;


 * sqlite:///country_club.db
Done.


facility_name,total_usage_by_members
Badminton Court,1086
Tennis Court 1,957
Massage Room 1,884
Tennis Court 2,882
Snooker Table,860
Pool Table,856
Table Tennis,794
Squash Court,418
Massage Room 2,54


In [89]:
%%sql
PRAGMA table_info(Facilities);


 * sqlite:///country_club.db
Done.


cid,name,type,notnull,dflt_value,pk
0,facid,INTEGER,0,None,0
1,name,TEXT,0,None,0
2,membercost,REAL,0,None,0
3,guestcost,REAL,0,None,0
4,initialoutlay,INTEGER,0,None,0
5,momthlymaintenance,INTEGER,0,None,0


In [91]:
#Find the facilities usage by month, but not guests
%%sql
SELECT 
    f.name AS facility_name,
    strftime('%Y-%m', b.starttime) AS month,
    SUM(b.slots) AS total_usage_by_members
FROM 
    Bookings b
JOIN 
    Facilities f ON b.facid = f.facid
WHERE 
    b.memid != 0
GROUP BY 
    f.name, month
ORDER BY 
    month, facility_name;


 * sqlite:///country_club.db
Done.


facility_name,month,total_usage_by_members
Badminton Court,None,1086
Massage Room 1,None,884
Massage Room 2,None,54
Pool Table,None,856
Snooker Table,None,860
Squash Court,None,418
Table Tennis,None,794
Tennis Court 1,None,957
Tennis Court 2,None,882
